## This Job will be uploaded to lambda and run twice a day

In [1]:
import time
import pythena
import datetime
import json
import s3fs
import pandas as pd
import nltk
from nltk.corpus import stopwords

athena_client = pythena.Athena("twitterappdatalake",region="ap-southeast-2") 


In [2]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    s3_cred_json=d["s3-access"]
    
##auth using s3fs
fs = s3fs.S3FileSystem(anon=False, key=s3_cred_json['ACCESS_KEY_ID'], secret=s3_cred_json['SECRET_ACCESS_KEY'],default_fill_cache=False)

In [5]:
fs.exists('s3://aggregatedresultsforapp/2020-09-08.csv' )

False

In [6]:
date_delete_list=[datetime.date.today(),datetime.date.today() - datetime.timedelta(days=1)]
for date_delete in date_delete_list:
    path='s3://aggregatedresultsforapp/' + date_delete.strftime("%Y-%m-%d") + '.csv'
    print(path)

s3://aggregatedresultsforapp/2020-08-08.csv
s3://aggregatedresultsforapp/2020-08-07.csv


In [5]:
## Custome Aggregation function. takes in numpy array
def word_freq(arr):
    stop_words = set(stopwords.words('english'))
    stop_substring=['#','@','&',"rt","https","https.","t.co"]
    stop_words.update(["rt","https","https.","-",'.',':'])
    text = " ".join(str(tweet).lower() for tweet in arr)
    all_freq={}
    for word in text.split():
        #Set the substring flag - if any substring is found then it won't get passed
        substring_flag=0
        for substring in stop_substring:
            if substring in word:
                substring_flag=1
        if substring_flag==0:
            if word not in stop_words:
                if word in all_freq: 
                    all_freq[word] += 1
                else: 
                    all_freq[word] = 1   
    word_freq_list=sorted(all_freq.items(), key=lambda x: x[1],reverse=True)[0:30]
    return word_freq_list

In [163]:
## Custome Aggregation function. takes in numpy array and returns the top tweets using the number of retweets to sort.
def top_tweets(df_partition):
    df_unique_tweets=df_partition.groupby('text').agg([('polarity','mean'),('retweet_count','max'),('retweet_count','count')]).reset_index()
    print(df_unique_tweets)
    df_sorted=df_unique_tweets.sort_values(by='max_retweet_count',ascending=False)
    df_head=df_sorted.head(20)
#     print(df_head)
    list_of_tweets=[]
    for index, row in df_head.iterrows():
        list_of_tweets.append({'max_retweet_count':row['max_retweet_count'],'text': str(row['text']),'count': row['count'],'avg_polarity': round(row['avg_polarity'],3)})
    return list_of_tweets

In [164]:
dftest.head(100).groupby(['partition_1','partition_0','search_query']).apply(top_tweets).to_frame()

SpecificationError: Function names must be unique, found multiple named retweet_count

In [3]:
## debug the funcitons
string_date='2020-07-20'
string_query='''
SELECT * FROM twitterdatalake t where date(date_parse(created_at,'%Y-%m-%d %H:%i:%s'))=date_parse('{0}','%Y-%m-%d')
'''.format(string_date)
df_oneday = athena_client.execute(string_query)
dftest=df_oneday[0]

In [157]:
# dftest2=dftest.groupby(['partition_1','partition_0','search_query']).apply(top_tweets).to_frame()
# dftest2.rename(columns={0:'top_tweets'})
df_top_tweets_groupby=dftest.groupby(['partition_1','partition_0','search_query']).apply(top_tweets).to_frame()
df_top_tweets_groupby=df_top_tweets_groupby.rename(columns={0:'top_tweets'})
df_word_freq_groupby=dftest.groupby(['partition_1','partition_0','search_query'])['text'].agg([word_freq,'count'])
df_avg_groupby=dftest.groupby(['partition_1','partition_0','search_query'])['polarity','followers','friends','retweet_count'].mean()
#join
df_final_agg=df_avg_groupby.join(df_word_freq_groupby, on=['partition_1','partition_0','search_query'])
df_final_agg=df_final_agg.join(df_top_tweets_groupby, on=['partition_1','partition_0','search_query'])
df_final_agg=df_final_agg.rename(columns={"text": "word_freq_list","polarity": "avg_polarity","followers":"avg_followers","friends":"avg_friends","retweet_count":"avg_retweet_count"})


In [161]:
df_final_agg

,,,avg_polarity,avg_followers,avg_friends,avg_retweet_count,word_freq,count,top_tweets
partition_1,partition_0,search_query,,,,,,,
AFC Bournemouth,EPL,#Bournemouth,-0.111374,8632.135908,1655.352408,12.893181,"[(fire, 806), (beach, 632), (bournemouth, 580)...",2097,"[{'max_retweet_count': 372, 'text': 'RT @footb..."
AMD,Technology,#AMD,0.095059,3551.473786,655.540777,5.585437,"[(amd, 174), (1, 118), (aorus, 100), (10, 99),...",1030,"[{'max_retweet_count': 25, 'text': '✨You ask f..."
ANU,Universities,#ANU,-0.072667,2719.515464,644.701031,1.659794,"[(last, 28), (years, 28), (attack, 28), (highl...",97,"[{'max_retweet_count': 4, 'text': 'Last years ..."
Adelaide,Australia,#Adelaide,0.126367,18152.342222,5688.822222,2.720000,"[(hot, 19), (sa, 18), (come, 18), (sauce, 16),...",225,"[{'max_retweet_count': 13, 'text': 'RT @Michae..."
Adidas,Fashion,#Adidas,0.365980,5215.033306,913.958784,52.506245,"[(check, 650), (good, 569), (items, 528), (lov...",2402,"[{'max_retweet_count': 30224, 'text': 'RT @GOT..."
...,...,...,...,...,...,...,...,...,...
Zambia,Politics,#Zambia,0.164025,39623.170833,3546.859524,13.276190,"[(new, 180), (president, 169), (business, 141)...",1680,"[{'max_retweet_count': 301, 'text': 'RT @bichr..."
Zara,Fashion,#Zara,0.465277,1310.962217,484.086902,39.754408,"[(check, 460), (good, 443), (share!, 443), (it...",794,"[{'max_retweet_count': 603, 'text': 'RT @Shukr..."
Zimbabwe,Politics,#Zimbabwe,-0.190760,6174.628506,1770.246399,203.620167,"[(arrest, 422), (journalist, 366), (hopewell, ...",1319,"[{'max_retweet_count': 1148, 'text': 'RT @NLin..."


In [128]:
def query_to_agg_df(string_date):
    string_query='''
    SELECT * FROM twitterdatalake t where date(date_parse(created_at,'%Y-%m-%d %H:%i:%s'))=date_parse('{0}','%Y-%m-%d')
    '''.format(string_date)
    df_oneday = athena_client.execute(string_query)
    dftest=df_oneday[0]
    df_word_freq_groupby=dftest.groupby(['partition_1','partition_0'])['text'].agg([word_freq,'count'])
    df_avg_groupby=dftest.groupby(['partition_1','partition_0'])['polarity','followers','friends','retweet_count'].mean()
    #join
    df_final_agg=df_avg_groupby.join(df_word_freq_groupby, on=['partition_1','partition_0'])
    df_final_agg=df_final_agg.rename(columns={"text": "word_freq_list","polarity": "avg_polarity","followers":"avg_followers","friends":"avg_friends","retweet_count":"avg_retweet_count"})
    
    return df_final_agg

In [129]:
for file in fs.listdir('aggregatedresultsforapp'):
    print(file)

{'Key': 'aggregatedresultsforapp/2020-07-22.csv', 'LastModified': datetime.datetime(2020, 7, 28, 10, 7, 4, tzinfo=tzutc()), 'ETag': '"b24abf1db77e66c95f8b53d9491fb7b8"', 'Size': 166774, 'StorageClass': 'STANDARD', 'type': 'file', 'size': 166774, 'name': 'aggregatedresultsforapp/2020-07-22.csv'}
{'Key': 'aggregatedresultsforapp/2020-07-23.csv', 'LastModified': datetime.datetime(2020, 7, 28, 10, 18, 39, tzinfo=tzutc()), 'ETag': '"707c34c8f79739f24c90f0258d552b69"', 'Size': 174512, 'StorageClass': 'STANDARD', 'type': 'file', 'size': 174512, 'name': 'aggregatedresultsforapp/2020-07-23.csv'}
{'Key': 'aggregatedresultsforapp/2020-07-24.csv', 'LastModified': datetime.datetime(2020, 7, 28, 10, 31, 55, tzinfo=tzutc()), 'ETag': '"beede18237886661bd10c5b11c0cf840"', 'Size': 164145, 'StorageClass': 'STANDARD', 'type': 'file', 'size': 164145, 'name': 'aggregatedresultsforapp/2020-07-24.csv'}


In [130]:
# string_today=today_date.strftime("%Y-%m-%d")
max_date=datetime.datetime.strptime("2020-07-13", "%Y-%m-%d").date()
#Find the max date in the s3 bucket
for file in fs.listdir('aggregatedresultsforapp'):
    current_string=file['Key'].split('/')[-1].split('.csv')[0]
    current_date=datetime.datetime.strptime(current_string, "%Y-%m-%d").date()
    if current_date>max_date:
        max_date=current_date

print(max_date)

2020-07-24


In [131]:
t0 = time.time()
day_before_date = datetime.date.today() - datetime.timedelta(days=2)
#If less than daybeforedate then ingest the next date else leave alone
if max_date<day_before_date:
    next_date=max_date + datetime.timedelta(days=1)
    str_next_date=next_date.strftime("%Y-%m-%d")
    df_final_agg=query_to_agg_df(str_next_date)
    file_dir='s3://aggregatedresultsforapp/'
    full_filename=file_dir+ str_next_date
    with fs.open(full_filename+ '.csv','w', encoding="utf-8", newline = '\n') as f:
        df_final_agg.to_csv(f)

t1 = time.time()
total = t1-t0
print(total)

37.00192832946777


In [133]:
len(df_final_agg)

284

In [ ]:
nltk.download_shell()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> c

Data Server:
  - URL: <https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml>
  - 7 Package Collections Available
  - 107 Individual Packages Available

Local Machine:
  - Data directory: C:\Users\William Jiang\AppData\Roaming\nltk_data

---------------------------------------------------------------------------
    s) Show Config   u) Set Server URL   d) Set Data Dir   m) Main Menu
---------------------------------------------------------------------------
